## Importamos Librerias

In [1]:
import requests
from datetime import datetime
import pandas as pd
import re

## Sacaos el id de cada categoría publicada

In [2]:
apiUrl = "https://tienda.mercadona.es/api/categories/"

In [ ]:
def obtener_ids_categorias_publicadas():
    response = requests.get(apiUrl)
    data = response.json()

    categorias_ids = []

    for categoria in data.get("results", []):
        if categoria.get("published", False):
            categorias_ids.append({
                "id": categoria["id"],
                "name": categoria["name"]
            })
        for subcategoria in categoria.get("categories", []):
            if subcategoria.get("published", False):
                categorias_ids.append({
                    "id": subcategoria["id"],
                    "name": subcategoria["name"]
                })
    return categorias_ids


def obtener_productos_por_categoria(categoria, fecha, hora):
    url = f"{apiUrl}{categoria['id']}"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error al obtener productos de categoría {categoria['id']}")
        return []
    
    data = response.json()
    productos_publicados = []
    for category in data.get("categories", []):
        for producto in category.get("products", []):
            if producto.get("published", False):
                price_info = producto.get("price_instructions", {})
                productos_publicados.append({
                    "id": producto["id"],
                    "name": producto["display_name"],
                    "price": float(price_info.get("unit_price") or 0),
                    "reference_price": float(price_info.get("bulk_price") or 0),
                    "reference_unit": price_info.get("reference_format"),
                    "category": categoria["name"],
                    "insert_date": fecha,
                    "insert_hour": hora
                })
    
    return productos_publicados

def limpiar_categoria(categoria):
    categoria = categoria.str.replace(' ', '_').str.lower()
    categoria = categoria.str.replace(',', '')
    ## remplazamos tildes por caracteres normales con regex
    categoria = categoria.apply(lambda x: re.sub(r'[áéíóú]', lambda m: {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u'}[m.group(0)], x))
    return categoria

def scrapear_productos():
    # Obtener la fecha y hora actual en variables separadas
    fecha_insercion_fecha = datetime.now().strftime("%Y-%m-%d")
    fecha_insercion_hora = datetime.now().strftime("%H:%M:%S")
    categorias = obtener_ids_categorias_publicadas()
    productos = []

    for categoria_id in categorias:
        productos_categoria = obtener_productos_por_categoria(categoria_id, fecha_insercion_fecha, fecha_insercion_hora)
        productos.extend(productos_categoria)
    print(f"Se han encontrado {len(productos)} productos en {len(categorias)} categorías.")
    # Convertir a DataFrame
    df_productos = pd.DataFrame(productos)
    df_categorias = pd.DataFrame(categorias)
    df_productos['category'] = limpiar_categoria(df_productos['category'])
    df_categorias['name'] = limpiar_categoria(df_categorias['name'])
    return df_productos, df_categorias

def concatenar_df(df1):
    df2 = pd.read_csv("productos_mercadona.csv", encoding='utf-8')
    
    # Limpiar valores con decimales y convertir solo a enteros
    def limpiar_id(id_value):
        try:
            # Convertir el id a un número flotante
            id_float = float(id_value)
            # Verificar si es entero (sin parte decimal)
            if id_float.is_integer():
                return int(id_float)  # Si es entero, devolver como int
            else:
                return None  # Si tiene decimales, devolver None
        except ValueError:
            return None  # En caso de que no se pueda convertir, devolver None

    # Aplicar la limpieza de IDs en ambos DataFrames
    df2['id'] = df2['id'].apply(limpiar_id)
    df1['id'] = df1['id'].apply(limpiar_id)

    # Eliminar las filas donde el id es None (valores no válidos)
    df2 = df2.dropna(subset=['id'])
    df1 = df1.dropna(subset=['id'])

    
    df_combinado = pd.concat([df1, df2], ignore_index=True)
    return df_combinado

def agregar_ecoicop_categorias(df_categorias, mapeo_ecoicop):
    """
    Añade la categoría ECOICOP al DataFrame de categorías de Mercadona.
    
    Args:
        df_categorias (pd.DataFrame): DataFrame con las categorías de Mercadona
        mapeo_ecoicop (dict): Diccionario que mapea categorías a grupos ECOICOP
        
    Returns:
        pd.DataFrame: DataFrame con la columna 'ecoicop_category' añadida
    """
    # Crear una copia para no modificar el original
    df = df_categorias.copy()
    
    # Aplicar el mapeo de categorías usando la columna 'name'
    df['ecoicop_category'] = df['name'].map(mapeo_ecoicop)
    
    # Asignar "Sin asignar" a las categorías que no tienen mapeo
    df['ecoicop_category'] = df['ecoicop_category'].fillna('Sin asignar')
    
    print(f"Categorías mapeadas: {(df['ecoicop_category'] != 'Sin asignar').sum()} de {len(df)}")
    print(f"Categorías sin mapear: {(df['ecoicop_category'] == 'Sin asignar').sum()}")
    
    return df

In [4]:
# Setteamos las categorias ECOICOP segun la categorías
mapeo_ecoicop = {
    # 01 Alimentos y bebidas no alcohólicas
    "aceite_vinagre_y_sal": "01 Alimentos y bebidas no alcohólicas",
    "especias": "01 Alimentos y bebidas no alcohólicas",
    "mayonesa_ketchup_y_mostaza": "01 Alimentos y bebidas no alcohólicas",
    "otras_salsas": "01 Alimentos y bebidas no alcohólicas",
    "agua": "01 Alimentos y bebidas no alcohólicas",
    "isotonico_y_energetico": "01 Alimentos y bebidas no alcohólicas",
    "refresco_de_cola": "01 Alimentos y bebidas no alcohólicas",
    "refresco_de_naranja_y_de_limon": "01 Alimentos y bebidas no alcohólicas",
    "tonica_y_bitter": "01 Alimentos y bebidas no alcohólicas",
    "refresco_de_te_y_sin_gas": "01 Alimentos y bebidas no alcohólicas",
    "aceitunas_y_encurtidos": "01 Alimentos y bebidas no alcohólicas",
    "frutos_secos_y_fruta_desecada": "01 Alimentos y bebidas no alcohólicas",
    "patatas_fritas_y_snacks": "01 Alimentos y bebidas no alcohólicas",
    "arroz": "01 Alimentos y bebidas no alcohólicas",
    "legumbres": "01 Alimentos y bebidas no alcohólicas",
    "pasta_y_fideos": "01 Alimentos y bebidas no alcohólicas",
    "azucar_y_edulcorante": "01 Alimentos y bebidas no alcohólicas",
    "chicles_y_caramelos": "01 Alimentos y bebidas no alcohólicas",
    "chocolate": "01 Alimentos y bebidas no alcohólicas",
    "golosinas": "01 Alimentos y bebidas no alcohólicas",
    "mermelada_y_miel": "01 Alimentos y bebidas no alcohólicas",
    "alimentacion_infantil": "01 Alimentos y bebidas no alcohólicas",
    "biberon_y_chupete": "01 Alimentos y bebidas no alcohólicas",
    "cerveza": "02 Bebidas alcohólicas y tabaco",
    "cerveza_sin_alcohol": "01 Alimentos y bebidas no alcohólicas",  # sin alcohol a 01
    "licores": "02 Bebidas alcohólicas y tabaco",
    "sidra_y_cava": "02 Bebidas alcohólicas y tabaco",
    "tinto_de_verano_y_sangria": "02 Bebidas alcohólicas y tabaco",
    "vino_blanco": "02 Bebidas alcohólicas y tabaco",
    "vino_lambrusco_y_espumoso": "02 Bebidas alcohólicas y tabaco",
    "vino_rosado": "02 Bebidas alcohólicas y tabaco",
    "vino_tinto": "02 Bebidas alcohólicas y tabaco",
    "cacao_soluble_y_chocolate_a_la_taza": "01 Alimentos y bebidas no alcohólicas",
    "cafe_capsula_y_monodosis": "01 Alimentos y bebidas no alcohólicas",
    "cafe_molido_y_en_grano": "01 Alimentos y bebidas no alcohólicas",
    "cafe_soluble_y_otras_bebidas": "01 Alimentos y bebidas no alcohólicas",
    "te_e_infusiones": "01 Alimentos y bebidas no alcohólicas",
    "arreglos": "01 Alimentos y bebidas no alcohólicas",  # supongo alimentos preparados
    "aves_y_pollo": "01 Alimentos y bebidas no alcohólicas",
    "carne_congelada": "01 Alimentos y bebidas no alcohólicas",
    "cerdo": "01 Alimentos y bebidas no alcohólicas",
    "conejo_y_cordero": "01 Alimentos y bebidas no alcohólicas",
    "embutido": "01 Alimentos y bebidas no alcohólicas",
    "hamburguesas_y_picadas": "01 Alimentos y bebidas no alcohólicas",
    "vacuno": "01 Alimentos y bebidas no alcohólicas",
    "empanados_y_elaborados": "01 Alimentos y bebidas no alcohólicas",
    "cereales": "01 Alimentos y bebidas no alcohólicas",
    "galletas": "01 Alimentos y bebidas no alcohólicas",
    "tortitas": "01 Alimentos y bebidas no alcohólicas",
    "aves_y_jamon_cocido": "01 Alimentos y bebidas no alcohólicas",
    "bacon_y_salchichas": "01 Alimentos y bebidas no alcohólicas",
    "chopped_y_mortadela": "01 Alimentos y bebidas no alcohólicas",
    "embutido_curado": "01 Alimentos y bebidas no alcohólicas",
    "jamon_serrano": "01 Alimentos y bebidas no alcohólicas",
    "pate_y_sobrasada": "01 Alimentos y bebidas no alcohólicas",
    "queso_curado_semicurado_y_tierno": "01 Alimentos y bebidas no alcohólicas",
    "queso_lonchas_rallado_y_en_porciones": "01 Alimentos y bebidas no alcohólicas",
    "queso_untable_y_fresco": "01 Alimentos y bebidas no alcohólicas",
    "arroz_y_pasta": "01 Alimentos y bebidas no alcohólicas",
    "carne": "01 Alimentos y bebidas no alcohólicas",
    "helados": "01 Alimentos y bebidas no alcohólicas",
    "hielo": "01 Alimentos y bebidas no alcohólicas",
    "marisco": "01 Alimentos y bebidas no alcohólicas",
    "pescado": "01 Alimentos y bebidas no alcohólicas",
    "pizzas": "01 Alimentos y bebidas no alcohólicas",
    "rebozados": "01 Alimentos y bebidas no alcohólicas",
    "tartas_y_churros": "01 Alimentos y bebidas no alcohólicas",
    "verdura": "01 Alimentos y bebidas no alcohólicas",
    "atun_y_otras_conservas_de_pescado": "01 Alimentos y bebidas no alcohólicas",
    "berberechos_y_mejillones": "01 Alimentos y bebidas no alcohólicas",
    "conservas_de_verdura_y_frutas": "01 Alimentos y bebidas no alcohólicas",
    "gazpacho_y_cremas": "01 Alimentos y bebidas no alcohólicas",
    "sopa_y_caldo": "01 Alimentos y bebidas no alcohólicas",
    "tomate": "01 Alimentos y bebidas no alcohólicas",
    "fruta": "01 Alimentos y bebidas no alcohólicas",
    "lechuga_y_ensalada_preparada": "01 Alimentos y bebidas no alcohólicas",
    "verdura": "01 Alimentos y bebidas no alcohólicas",
    "huevos": "01 Alimentos y bebidas no alcohólicas",
    "leche_y_bebidas_vegetales": "01 Alimentos y bebidas no alcohólicas",
    "mantequilla_y_margarina": "01 Alimentos y bebidas no alcohólicas",
    "bolleria_de_horno": "01 Alimentos y bebidas no alcohólicas",
    "bolleria_envasada": "01 Alimentos y bebidas no alcohólicas",
    "harina_y_preparado_reposteria": "01 Alimentos y bebidas no alcohólicas",
    "pan_de_horno": "01 Alimentos y bebidas no alcohólicas",
    "pan_de_molde_y_otras_especialidades": "01 Alimentos y bebidas no alcohólicas",
    "pan_tostado_y_rallado": "01 Alimentos y bebidas no alcohólicas",
    "picos_rosquilletas_y_picatostes": "01 Alimentos y bebidas no alcohólicas",
    "tartas_y_pasteles": "01 Alimentos y bebidas no alcohólicas",
    "listo_para_comer": "01 Alimentos y bebidas no alcohólicas",
    "pizzas": "01 Alimentos y bebidas no alcohólicas",
    "platos_preparados_calientes": "01 Alimentos y bebidas no alcohólicas",
    "platos_preparados_frios": "01 Alimentos y bebidas no alcohólicas",
    "bifidus": "01 Alimentos y bebidas no alcohólicas",
    "flan_y_natillas": "01 Alimentos y bebidas no alcohólicas",
    "gelatina_y_otros_postres": "01 Alimentos y bebidas no alcohólicas",
    "postres_de_soja": "01 Alimentos y bebidas no alcohólicas",
    "yogures_desnatados": "01 Alimentos y bebidas no alcohólicas",
    "yogures_griegos": "01 Alimentos y bebidas no alcohólicas",
    "yogures_liquidos": "01 Alimentos y bebidas no alcohólicas",
    "yogures_naturales_y_sabores": "01 Alimentos y bebidas no alcohólicas",
    "yogures_y_postres_infantiles": "01 Alimentos y bebidas no alcohólicas",
    "fruta_variada": "01 Alimentos y bebidas no alcohólicas",
    "melocoton_y_piña": "01 Alimentos y bebidas no alcohólicas",
    "naranja": "01 Alimentos y bebidas no alcohólicas",
    "tomate_y_otros_sabores": "01 Alimentos y bebidas no alcohólicas",
    # 06 Sanidad (productos higiene y cuidado personal)
    "higiene_y_cuidado": "06 Sanidad",
    "toallitas_y_pañales": "06 Sanidad",
    "acondicionador_y_mascarilla": "06 Sanidad",
    "champu": "06 Sanidad",
    "coloracion_cabello": "06 Sanidad",
    "fijacion_cabello": "06 Sanidad",
    "afeitado_y_cuidado_para_hombre": "06 Sanidad",
    "cuidado_corporal": "06 Sanidad",
    "cuidado_e_higiene_facial": "06 Sanidad",
    "depilacion": "06 Sanidad",
    "desodorante": "06 Sanidad",
    "gel_y_jabon_de_manos": "06 Sanidad",
    "higiene_bucal": "06 Sanidad",
    "higiene_intima": "06 Sanidad",
    "manicura_y_pedicura": "06 Sanidad",
    "perfume_y_colonia": "06 Sanidad",
    "protector_solar_y_aftersun": "06 Sanidad",
    "fitoterapia": "06 Sanidad",
    "parafarmacia": "06 Sanidad",
    # 12 Otros bienes y servicios (limpieza, hogar, mascotas)
    "detergente_y_suavizante_ropa": "12 Otros bienes y servicios",
    "estropajo_bayeta_y_guantes": "12 Otros bienes y servicios",
    "insecticida_y_ambientador": "12 Otros bienes y servicios",
    "lejia_y_liquidos_fuertes": "12 Otros bienes y servicios",
    "limpiacristales": "12 Otros bienes y servicios",
    "limpiahogar_y_friegasuelos": "12 Otros bienes y servicios",
    "limpieza_baño_y_wc": "12 Otros bienes y servicios",
    "limpieza_cocina": "12 Otros bienes y servicios",
    "limpieza_muebles_y_multiusos": "12 Otros bienes y servicios",
    "limpieza_vajilla": "12 Otros bienes y servicios",
    "menaje_y_conservacion_de_alimentos": "12 Otros bienes y servicios",
    "papel_higienico_y_celulosa": "12 Otros bienes y servicios",
    "pilas_y_bolsas_de_basura": "12 Otros bienes y servicios",
    "utensilios_de_limpieza_y_calzado": "12 Otros bienes y servicios",
    
    # 01 Alimentos (mascotas)
    "gato": "01 Alimentos y bebidas no alcohólicas",
    "perro": "01 Alimentos y bebidas no alcohólicas",
    "otros": "12 Otros bienes y servicios",
}

In [ ]:
# Scrapeamos los productos y categorías
df_productos, df_categorias = scrapear_productos()
#lo unimos con el antiguo dataframe
df_mercadona = concatenar_df(df_productos)
##
# Concatenamos los GRUPOS ECOICOP del INE para correlacionar en PowerBI
df_categorias_ecoicop = agregar_ecoicop_categorias(df_categorias, mapeo_ecoicop)

df_mercadona.to_csv("productos_mercadona.csv", index=False, encoding='utf-8', sep=',', decimal='.')
df_categorias_ecoicop.to_csv("categorias_y_ecoicop.csv", index=False, encoding='utf-8')

print("Los datos han sido guardados en productos_mercadona.csv y categorias_mercadona.csv")
# Forzar los precios a texto con punto decimal, para que Power BI los lea correctamente
df_mercadona['price'] = df_mercadona['price'].apply(lambda x: format(x, '.2f'))
df_mercadona['reference_price'] = df_mercadona['reference_price'].apply(lambda x: format(x, '.2f'))

# Este es el DataFrame que Power BI va a importar
df_mercadona



Se han encontrado 4791 productos en 151 categorías.
Categorías mapeadas: 142 de 151
Categorías sin mapear: 9
Los datos han sido guardados en productos_mercadona.csv y categorias_mercadona.csv


,id,name,price,reference_price,reference_unit,category,insert_date,insert_hour
0,4241.0,"Aceite de oliva 0,4º Hacendado",18.95,3.79,L,aceite_vinagre_y_sal,2025-05-20,13:41:39
1,4240.0,"Aceite de oliva 0,4º Hacendado",4.20,4.20,L,aceite_vinagre_y_sal,2025-05-20,13:41:39
2,4717.0,Aceite de oliva virgen extra Hacendado,14.95,4.98,L,aceite_vinagre_y_sal,2025-05-20,13:41:39
3,4740.0,Aceite de oliva virgen extra Hacendado,5.25,5.25,L,aceite_vinagre_y_sal,2025-05-20,13:41:39
4,4706.0,Aceite de oliva virgen extra Hacendado Gran Se...,5.95,7.93,L,aceite_vinagre_y_sal,2025-05-20,13:41:39
5,4641.0,Aceite de oliva 1º Hacendado,18.95,3.79,L,aceite_vinagre_y_sal,2025-05-20,13:41:39
6,4640.0,Aceite de oliva 1º Hacendado,4.20,4.20,L,aceite_vinagre_y_sal,2025-05-20,13:41:39
7,4711.0,Aceite de oliva virgen Hacendado,12.70,4.23,L,aceite_vinagre_y_sal,2025-05-20,13:41:39
8,4749.0,Aceite de oliva virgen Hacendado,4.65,4.65,L,aceite_vinagre_y_sal,2025-05-20,13:41:39
9,4718.0,Aceite de oliva virgen extra Hacendado,2.80,14.00,L,aceite_vinagre_y_sal,2025-05-20,13:41:39


## Script visual de python

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Normalizar precio de referencia
def normalize_units(row):
    unit = row['reference_unit']
    if unit == 'L':
        row['reference_price'] *= 1000
    elif unit == 'kg':
        row['reference_price'] *= 1000
    elif unit == '100 g':
        row['reference_price'] *= 10
    elif unit == '100 ml':
        row['reference_price'] *= 10
    elif unit == 'dc':
        row['reference_price'] *= 100
    elif unit == 'm':
        row['reference_price'] *= 1000
    return row

dataset = dataset.apply(normalize_units, axis=1)

# Seleccionar los 10 productos más caros
top10 = dataset.sort_values('price', ascending=False).head(10)

# Gráfico de barras
plt.figure(figsize=(10, 6))
sns.barplot(data=top10, x='price', y='name', hue='category', dodge=False)
plt.title('Top 10 productos más caros')
plt.xlabel('Precio')
plt.ylabel('Producto')
plt.tight_layout()
plt.show()
